In [32]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from PIL import Image, ImageDraw
import math

class Vehicle:
    def __init__(self):
        self.lw = 2.8*10  # wheelbase
        self.lf = 0.96*10  # front hang length
        self.lr = 0.929*10  # rear hang length
        self.lb = 1.942*10  # width

    def create_polygon(self, x, y, theta):
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)

        points = np.array([
            [-self.lr, -self.lb / 2, 1],
            [self.lf + self.lw, -self.lb / 2, 1],
            [self.lf + self.lw, self.lb / 2, 1],
            [-self.lr, self.lb / 2, 1],
            [-self.lr, -self.lb / 2, 1],
        ]).dot(np.array([
            [cos_theta, -sin_theta, x],
            [sin_theta, cos_theta, y],
            [0, 0, 1]
        ]).transpose())
        return points[:, 0:2]


class Case:
    def __init__(self):
        self.x_s, self.y_s, self.theta_s = 0, 0, 0
        self.x_g, self.y_g, self.theta_g = 0, 0, 0
        self.xmin, self.xmax = 0, 0
        self.ymin, self.ymax = 0, 0
        self.obs_num = 0
        self.obs = np.array([])
        self.vehicle = Vehicle()

    @staticmethod
    def read(file):
        case = Case()
        multiple=10
        lw=2.8*multiple
        with open(file, 'r') as f:
            reader = csv.reader(f)
            tmp = list(reader)
            v = [float(i)*multiple for i in tmp[0]]
            case.x_s, case.y_s, case.theta_s = v[0:3]
            case.x_g, case.y_g, case.theta_g = v[3:6]
            case.theta_s = v[2]/multiple
            case.theta_g = v[5]/multiple
            case.xmin = min(case.x_s, case.x_g) - 8*multiple
            case.xmax = max(case.x_s, case.x_g) + 8*multiple
            case.ymin = min(case.y_s, case.y_g) - 8*multiple
            case.ymax = max(case.y_s, case.y_g) + 8*multiple
            case.x_s-=case.xmin
            case.y_s-=case.ymin
            case.x_g-=case.xmin
            case.y_g-=case.ymin
            case.x_s = case.x_s+math.cos(case.theta_s)*lw/2
            case.y_s = case.y_s+math.sin(case.theta_s)*lw/2
            case.x_g = case.x_g+math.cos(case.theta_g)*lw/2
            case.y_g = case.y_g+math.sin(case.theta_g)*lw/2
            case.obs_num = int(v[6]/multiple)
            num_vertexes = np.array(v[7:7 + case.obs_num], dtype=np.int)
            num_vertexes = num_vertexes // multiple
            vertex_start = 7 + case.obs_num + (np.cumsum(num_vertexes, dtype=np.int) - num_vertexes) * 2
            case.obs = []
            for vs, nv in zip(vertex_start, num_vertexes):
                case.obs.append(np.array(v[vs:vs + nv * 2]).reshape((nv, 2), order='A'))
            for array in case.obs:
                array[:, 0] = array[:, 0] - case.xmin # 对 X 值乘以 2
                array[:, 1] = array[:, 1] - case.ymin  # 对 Y 值加 3
        return case



def main():
    for i in range(0, 20):
        case = Case.read('Case%d.csv' % (i + 1))
        width, height = case.xmax-case.xmin, case.ymax-case.ymin
        image = Image.new('RGB', (int(width), int(height)), 'white')
        draw = ImageDraw.Draw(image)
    # points=[(-30.45737672 ,20.63054671),(100.47540218,80.18731484),(110.19481843,60.3834844),(-20.73796048,8.82671627)]
        for array in case.obs:
            array_list = array.tolist()
            points = [(x, y) for x, y in array_list]
            draw.polygon(points, outline='black', fill='black')

        
        image.save('TPCAP_%d.png'%(i+1))
        print(str(case.x_s)+" "+str(height-case.y_s)+" "+str(6.28-case.theta_s)+" \n"+str(case.x_g)+" "+str(height-case.y_g)+" "+str(6.28-case.theta_g))
        


if __name__ == "__main__":
    main()

/var/folders/9p/dm8ssc490bvf953k5btv8gh00000gn/T/ipykernel_37564/3853736460.py:68: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num_vertexes = np.array(v[7:7 + case.obs_num], dtype=np.int)
/var/folders/9p/dm8ssc490bvf953k5btv8gh00000gn/T/ipykernel_37564/3853736460.py:70: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If yo

93.71982247414874 77.21316105993854 6.079601446174122 
139.2725836936823 87.25149402765437 5.900505256331101
87.68500485431825 91.70216648271187 7.26971402799757 
122.96952303299064 203.67372252393318 5.518549353524759
88.56621009878758 91.07339354233514 7.192370953011526 
113.7503424885704 173.47744450617807 6.1334081442083415
78.08707644008162 93.86869580940862 7.987862501105081 
105.9434764865351 110.02906109503421 8.20854240726007
67.96203358622972 127.578825601028 3.6742185843706703 
125.22168005820045 93.66661884257533 8.06946527266884
178.81154884699947 151.74345778887562 4.55260179622309 
93.24071811580657 84.54789883108137 6.610853033811846
137.62591158296843 68.1013623160267 5.26419940054369 
86.83090353738781 101.11290957378633 5.21891086733199
93.5913486701897 112.71109644348091 6.522208587109621 
175.3407135409799 93.5119645887796 8.11561365670069
283.36066730946857 90.75569684226646 5.784448326514172 
90.75509791436326 71.03741840469729 5.585261723803297
70.56738893384536

In [1]:
import cv2

# 读取图像
image = cv2.imread('TPCAP_8.png')

# 设置模糊核的大小
kernel_size = (5, 5)  # 这里可以根据需要调整核的大小

# 应用模糊滤波
blurred_image = cv2.blur(image, kernel_size)

cv2.imwrite('blurred_imag5.png', blurred_image)

# 显示原始图像和模糊图像
cv2.imshow('Original Image', image)
cv2.imshow('Blurred Image', blurred_image)

# 等待按键后关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()


2023-12-11 12:56:21.259 python[35653:1410759] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-12-11 12:56:21.259 python[35653:1410759] Text input context does not respond to _valueForTIProperty:
2023-12-11 12:56:21.261 python[35653:1410759] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-12-11 12:56:21.261 python[35653:1410759] Text input context does not respond to _valueForTIProperty:
2023-12-11 12:56:23.483 python[35653:1410759] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-12-11 12:56:23.483 python[35653:1410759] Text input context does not respond to _valueForTIProperty:
